## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [1]:
DATA_URL = "http://az.lib.ru/l/leskow_n_s/text_0246.shtml"

Устанавливаем библиотеки

In [2]:
import warnings
warnings.filterwarnings('ignore')
!pip -q install rnnmorph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 69.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 94.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Создаём объект морфологического анализатора `RNNMorph`

In [3]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [4]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [5]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Лесков Николай Семенович. Левша</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/l/leskow_n_s/>Лесков Николай Семенович</a><br>\r\nЛевша</h2>\r\n\r\n<!-------'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [6]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

for script in soup(["script", "style"]):
    script.extract()

cleaned_text = soup.get_text()

In [7]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Лесков Николай Семенович. Левша\n\n\n\nЛесков Николай Семенович\r\nЛевша\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Помощь]\r\n\r\n'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [8]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


"A total of 466 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [18]:
from tqdm import tqdm
raw_predictions = [[pred.normal_form  for pred in sent]
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]


8/8 [==============================] - 7s 868ms/step



sentences: 100%|██████████| 466/466 [00:00<00:00, 52741.46it/s]


In [24]:
predictions = [[pred for pred in sent if str(pred).isalpha() ] for sent in raw_predictions]

predictions[0:5]

[['лесок', 'николай', 'семёнович'],
 ['левша',
  'лесок',
  'николай',
  'семёнович',
  'левша',
  'регистрация',
  'найти',
  'рейтинг',
  'обсуждение',
  'новинка',
  'обзор',
  'помощь',
  'комментарий',
  'последний',
  'от'],
 ['лесок', 'николай', 'семёнович', 'yes', 'год', 'обновлённый'],
 [],
 ['статистика']]

Проверьте количество предложений и токенов

In [25]:
len(predictions)

466

In [26]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens)


9053

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 150 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам.

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75.

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [34]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
nltk.download("stopwords")
stopwords_set = set(stopwords.words("russian"))
stopwords.words("russian")[:5] #Пример стоп слов

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['и', 'в', 'во', 'не', 'что']

In [36]:
from collections import Counter

word_count = Counter(non_uniq_tokens)
parasites = 0
for word, count in word_count.most_common()[:150]:
    if word in stopwords_set:
      parasites += 1
ans = (150 - parasites) / 150
ans

0.5333333333333333

Получилось, что только 53% слов из 150 самых частых не являются стопсловами

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 10 раз.

In [37]:
amount_more10_words = 0
for word, count in word_count.most_common():
  if count > 10:
    amount_more10_words += 1
amount_more10_words

126

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
